# Polis Data Project

# 0. Setup

In [1]:
%pip install docling
%pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import cv2
import numpy as np
import torch
import re
import csv

from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions

# 1. Pre-processing and conversion export

#### Load source document and specify conversion format

In [ ]:
# source = "[corpus file].pdf" #
source = "HN_sample.pdf"

converter = DocumentConverter()
doc = converter.convert(source)

# Choose conversion format (markdown default) #
text = doc.document.export_to_markdown()
#text = doc.document.export_to_text()

# raw text check #
print(text)

2026-01-05 01:16:02,604 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-05 01:16:02,611 - INFO - Going to convert document batch...
2026-01-05 01:16:02,612 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2026-01-05 01:16:02,614 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2026-01-05 01:16:02,615 - INFO - easyocr cannot be used because it is not installed.
2026-01-05 01:16:02,615 - INFO - Accelerator device: 'cpu'
[INFO] 2026-01-05 01:16:02,646 [RapidOCR] base.py:22: Using engine_name: torch


[INFO] 2026-01-05 01:16:02,647 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-05 01:16:02,665 [RapidOCR] download_file.py:60: File exists and is valid: /work/PDP/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-05 01:16:02,665 [RapidOCR] main.py:50: Using /work/PDP/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-05 01:16:02,971 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-05 01:16:02,973 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-05 01:16:02,978 [RapidOCR] download_file.py:60: File exists and is valid: /work/PDP/.venv/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-05 01:16:02,979 [RapidOCR] main.py:50: Using /work/PDP/.venv/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-05 01:16:03,103 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-05 

#### Fix character errors

In [38]:
# Fix missing numbers #
def fix_digits(text):
    digit_map = str.maketrans({
        '': '0', '': '1', '': '2', '': '3', '': '4',
        '': '5', '': '6', '': '7', '': '8', '': '9'
    })
    return text.translate(digit_map)
text = fix_digits(text)

# Fix "lenis" corruption #
def fix_lenis(text):
    # Remove 'lenis' that appears within Greek text
    text = re.sub(r'lenis', '', text)
    # Remove forward slashes that appear in Greek patterns
    text = re.sub(r'/([Α-Ωα-ω])', r'\1', text)
    return text
text = fix_lenis(text)

# Fix Greek letters #
greek_map = {
    '/Alpha': 'Α', '/alpha': 'α', '/alphaacute': 'ά',
    '/Beta': 'Β', '/beta': 'β',
    '/Gamma': 'Γ', '/gamma': 'γ',
    '/Delta': 'Δ', '/delta': 'δ',
    '/Epsilon': 'Ε', '/epsilon': 'ε', '/epsilonacute': 'έ',
    '/Zeta': 'Ζ', '/zeta': 'ζ',
    '/Eta': 'Η', '/eta': 'η', '/etaacute': 'ή',
    '/Theta': 'Θ', '/theta': 'θ',
    '/Iota': 'Ι', '/iota': 'ι', '/iotaacute': 'ί', '/iotatilde': 'ῖ',
    '/Kappa': 'Κ', '/kappa': 'κ',
    '/Lambda': 'Λ', '/lambda': 'λ',
    '/Mu': 'Μ', '/mu': 'μ',
    '/Nu': 'Ν', '/nu': 'ν',
    '/Xi': 'Ξ', '/xi': 'ξ',
    '/Omicron': 'Ο', '/omicron': 'ο', '/omicronacute': 'ό',
    '/Pi': 'Π', '/pi': 'π',
    '/Rho': 'Ρ', '/rho': 'ρ',
    '/Sigma': 'Σ', '/sigma': 'σ', '/sigmaf': 'ς',
    '/Tau': 'Τ', '/tau': 'τ',
    '/Upsilon': 'Υ', '/upsilon': 'υ',
    '/Phi': 'Φ', '/phi': 'φ',
    '/Chi': 'Χ', '/chi': 'χ',
    '/Psi': 'Ψ', '/psi': 'ψ',
    '/Omega': 'Ω', '/omega': 'ω', '/omegatilde': 'ῶ', '/omegaacute': 'ώ',
}
def fix_greek(text):
    # Sort by length (longest first) to avoid partial replacements
    for greek_code, greek_char in sorted(greek_map.items(), key=lambda x: len(x[0]), reverse=True):
        text = text.replace(greek_code, greek_char)
    return text
text = fix_greek(text)

# Text check #
print(text)

Massalia  (no. 3 )  itself, a  foundation  of c. 600 . However, archaeological evidence suggests the possibility of pre-colonial contacts between Greeks and indigenous populations, e.g. the three cups and the oenochoe of C 7 l from the indigenous cemetery of Le Peyrou, near the place where Agathe ( infra )  was  later  founded  (Nickels et  al . ( 1981 ); Nickels ( 1989 )), and  the  sporadic  finds  of C 7 l  Greek  pottery  at Mailhac (Gailledrat ( 1997 ) 69 -70 ).

However, the  foundation  of Massalia  marked  the  real beginning of Greek presence and expansion in the region, an  expansion  that  was  basically  a  coastal  phenomenon; besides, the expansion of Massalia was a long-term process running from C 6 to the beginnings of the Roman occupation  of the  region. Massaliote  activity, moreover, varied from period to period: during C 6 and C 5 e it was directed towards  the  development  of commercial  interests  (Bats ( 1988 )), whereas from C 5 l/C 4 e it aimed primarily at e

In [48]:
def clean_text(text):
    # Fix digits
    digit_map = str.maketrans({
        '': '0', '': '1', '': '2', '': '3', '': '4',
        '': '5', '': '6', '': '7', '': '8', '': '9'
    })
    text = text.translate(digit_map)
    
    # Fix lenis corruption
    text = re.sub(r'lenis', '', text)
    text = re.sub(r'/([Α-Ωα-ω])', r'\1', text)
    
    # Fix Greek letters
    greek_map = {
        '/Alpha': 'Α', '/alpha': 'α', '/alphaacute': 'ά',
        '/Beta': 'Β', '/beta': 'β',
        '/Gamma': 'Γ', '/gamma': 'γ',
        '/Delta': 'Δ', '/delta': 'δ',
        '/Epsilon': 'Ε', '/epsilon': 'ε', '/epsilonacute': 'έ',
        '/Zeta': 'Ζ', '/zeta': 'ζ',
        '/Eta': 'Η', '/eta': 'η', '/etaacute': 'ή',
        '/Theta': 'Θ', '/theta': 'θ',
        '/Iota': 'Ι', '/iota': 'ι', '/iotaacute': 'ί', '/iotatilde': 'ῖ',
        '/Kappa': 'Κ', '/kappa': 'κ',
        '/Lambda': 'Λ', '/lambda': 'λ',
        '/Mu': 'Μ', '/mu': 'μ',
        '/Nu': 'Ν', '/nu': 'ν',
        '/Xi': 'Ξ', '/xi': 'ξ',
        '/Omicron': 'Ο', '/omicron': 'ο', '/omicronacute': 'ό',
        '/Pi': 'Π', '/pi': 'π',
        '/Rho': 'Ρ', '/rho': 'ρ',
        '/Sigma': 'Σ', '/sigma': 'σ', '/sigmaf': 'ς',
        '/Tau': 'Τ', '/tau': 'τ',
        '/Upsilon': 'Υ', '/upsilon': 'υ',
        '/Phi': 'Φ', '/phi': 'φ',
        '/Chi': 'Χ', '/chi': 'χ',
        '/Psi': 'Ψ', '/psi': 'ψ',
        '/Omega': 'Ω', '/omega': 'ω', '/omegatilde': 'ῶ', '/omegaacute': 'ώ',
    }
    
    for greek_code, greek_char in sorted(greek_map.items(), key=lambda x: len(x[0]), reverse=True):
        text = text.replace(greek_code, greek_char)
    
    return text

# Clean the text
text = clean_text(text)

# Check a sample line
print("=== SAMPLE CHECK ===")
sample = [line for line in text.split('\n') if 'Agathe' in line]
if sample:
    print(sample[0][:2000])

=== SAMPLE CHECK ===
Massalia  (no. 3 )  itself, a  foundation  of c. 600 . However, archaeological evidence suggests the possibility of pre-colonial contacts between Greeks and indigenous populations, e.g. the three cups and the oenochoe of C 7 l from the indigenous cemetery of Le Peyrou, near the place where Agathe ( infra )  was  later  founded  (Nickels et  al . ( 1981 ); Nickels ( 1989 )), and  the  sporadic  finds  of C 7 l  Greek  pottery  at Mailhac (Gailledrat ( 1997 ) 69 -70 ).


#### Text checks

In [6]:
print(f"Document character length: {len(text)}")
print(f"Word count (approximate): {len(text.split())}")
print(f"Number of lines: {len(text.splitlines())}")
print(f"Text sample: {text[:1000]}")

Document character length: 16776
Word count (approximate): 3146
Number of lines: 53
Text sample: Massalia  (no. 3 )  itself, a  foundation  of c. 600 . However, archaeological evidence suggests the possibility of pre-colonial contacts between Greeks and indigenous populations, e.g. the three cups and the oenochoe of C 7 l from the indigenous cemetery of Le Peyrou, near the place where Agathe ( infra )  was  later  founded  (Nickels et  al . ( 1981 ); Nickels ( 1989 )), and  the  sporadic  finds  of C 7 l  Greek  pottery  at Mailhac (Gailledrat ( 1997 ) 69 -70 ).

However, the  foundation  of Massalia  marked  the  real beginning of Greek presence and expansion in the region, an  expansion  that  was  basically  a  coastal  phenomenon; besides, the expansion of Massalia was a long-term process running from C 6 to the beginnings of the Roman occupation  of the  region. Massaliote  activity, moreover, varied from period to period: during C 6 and C 5 e it was directed towards  the  develop

#### Export to new markdown file

In [39]:
with open('HN_sample.md', 'w', encoding='utf-8') as f:
    f.write(text)

print("text saved as markdown")

text saved as markdown


# 2. Sectioning / Serialization

# 3. Extract and update .csv

#### City names

In [40]:
city_pattern = r'^([A-Z][^\n(]+?)\s+\('

cities = re.findall(city_pattern, text, re.MULTILINE)

# Clean up the city names (remove extra whitespace)
cities = [city.strip() for city in cities]

print("=== CITIES FOUND ===")
for i, city in enumerate(cities, 1):
    print(f"{i}. {city}")

# Write to CSV
with open('PDP.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['City'])
    for city in cities:
        writer.writerow([city])

print(f"\n✓ Extracted {len(cities)} cities to PDP.csv")

=== CITIES FOUND ===
1. Massalia
2. However, the  foundation  of Massalia  marked  the  real beginning of Greek presence and expansion in the region, an  expansion  that  was  basically  a  coastal  phenomenon; besides, the expansion of Massalia was a long-term process running from C 6 to the beginnings of the Roman occupation  of the  region. Massaliote  activity, moreover, varied from period to period: during C 6 and C 5 e it was directed towards  the  development  of commercial  interests
3. Agathe
4. Antipolis
5. Avenion
6. Azania
7. Kabellion
8. Kyrene
9. Monoikos
10. Nikaia
11. Olbia
12. Rhodanousia
13. Sekoanos
14. Stoichades Islands
15. Tauroeis
16. Troizen
17. The Greeks referred to the coast of the Iberian peninsula by the generic name ' Ιβηρία , a term which, as time went on, ended up as the name of the whole peninsula
18. If the  late  and  hardly  reliable  references  to  Rhodian colonisation in Iberia
19. Alonis
20. Hemeroskopeion

✓ Extracted 20 cities to PDP.csv


In [41]:
import re
import csv

# Patterns
city_pattern = r'^([A-Z][^\n(]+?)\s+\('
greek_pattern = r'\(([\u0370-\u03FF\u1F00-\u1FFF][^)]*)\)'

# Extract cities and their Greek names
results = []
for line in text.split('\n'):
    city_match = re.match(city_pattern, line)
    if city_match:
        city_name = city_match.group(1).strip()
        
        # Look for Greek text in the same line
        greek_match = re.search(greek_pattern, line)
        if greek_match:
            # Only include if Greek name is found (filters out false positives)
            greek_name = greek_match.group(1).strip()
            results.append((city_name, greek_name))

print("=== CITIES FOUND ===")
for i, (city, greek) in enumerate(results, 1):
    print(f"{i}. {city} ({greek})")

# Write to CSV
with open('PDP.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['City', 'Greek_Name'])
    for city, greek in results:
        # Clean up asterisks from city names
        city_clean = city.replace('*', '')
        writer.writerow([city_clean, greek])

print(f"\n✓ Extracted {len(results)} cities to PDP.csv")

=== CITIES FOUND ===

✓ Extracted 0 cities to PDP.csv


In [45]:
# Patterns
city_pattern = r'^([A-Z][^\n(]+?)\s+\('
greek_pattern = r'\(([\u0370-\u03FF\u1F00-\u1FFF][^)]*)\)'

# Extract cities and their Greek names
results = []
for line in cleaned_text.split('\n'):
    city_match = re.match(city_pattern, line)
    if city_match:
        city_name = city_match.group(1).strip()
        
        # Look for Greek text in the same line
        greek_match = re.search(greek_pattern, line)
        if greek_match:
            greek_name = greek_match.group(1).strip()
            results.append((city_name, greek_name))

print("=== CITIES FOUND ===")
for i, (city, greek) in enumerate(results, 1):
    print(f"{i}. {city} ({greek})")

# Write to CSV
with open('PDP.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['City', 'Greek_Name'])
    for city, greek in results:
        city_clean = city.replace('*', '')
        writer.writerow([city_clean, greek])

print(f"\n✓ Extracted {len(results)} cities to PDP.csv")

=== CITIES FOUND ===

✓ Extracted 0 cities to PDP.csv


In [46]:
# Debug: Check what's in cleaned_text
print("=== DEBUG: Text Sample ===")
print(cleaned_text[:500])
print("\n=== DEBUG: Check for Greek characters ===")
greek_found = re.findall(r'[\u0370-\u03FF\u1F00-\u1FFF]+', cleaned_text)
print(f"Greek strings found: {len(greek_found)}")
if greek_found:
    print("First 5:", greek_found[:5])

print("\n=== DEBUG: Lines starting with capitals ===")
lines_with_caps = [line for line in cleaned_text.split('\n') if line and line[0].isupper()]
print(f"Found {len(lines_with_caps)} lines")
if lines_with_caps:
    print("\nFirst 3 lines:")
    for line in lines_with_caps[:3]:
        print(line[:150])

print("\n=== DEBUG: Lines with parentheses ===")
lines_with_parens = [line for line in cleaned_text.split('\n') if '(' in line and line and line[0].isupper()]
print(f"Found {len(lines_with_parens)} lines")
if lines_with_parens:
    print("\nFirst 3:")
    for line in lines_with_parens[:3]:
        print(line[:150])
        

=== DEBUG: Text Sample ===
Massalia  (no. 3 )  itself, a  foundation  of c. 600 . However, archaeological evidence suggests the possibility of pre-colonial contacts between Greeks and indigenous populations, e.g. the three cups and the oenochoe of C 7 l from the indigenous cemetery of Le Peyrou, near the place where Agathe ( infra )  was  later  founded  (Nickels et  al . ( 1981 ); Nickels ( 1989 )), and  the  sporadic  finds  of C 7 l  Greek  pottery  at Mailhac (Gailledrat ( 1997 ) 69 -70 ).

However, the  foundation  o

=== DEBUG: Check for Greek characters ===
Greek strings found: 119
First 5: ['Αγάθη', 'πόλεις', 'Ελληνίδες', 'Μασσαλιῶται', 'Φωκαεῖς']

=== DEBUG: Lines starting with capitals ===
Found 22 lines

First 3 lines:
Massalia  (no. 3 )  itself, a  foundation  of c. 600 . However, archaeological evidence suggests the possibility of pre-colonial contacts between Gree
However, the  foundation  of Massalia  marked  the  real beginning of Greek presence and expansion in the reg